In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [32]:
!pip install sentence-transformers==2.1.0
!pip install pandas==1.2.2
!pip install umap-learn==0.5.2

     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 38.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82708 sha256=acfbc08634fddfdc1dc96b8e5d2bc9f03d6b9d44b18912e87baa4908edc85b05
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=201873ab3f10fc38b3aed661801ac1f82d7b245ca3170f6ccf20c78948a1400a
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
Successfully built umap-learn pynndescent


In [3]:
!nvidia-smi

Wed Mar  9 13:37:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Импорты

In [33]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import umap

In [29]:
from pickle import dump, load, HIGHEST_PROTOCOL
from typing import Any


class Pickler:
    @staticmethod
    def pickle(object_to_pickle: Any, file: str) -> None:
        with open(file, 'wb') as handle:
            dump(object_to_pickle, handle, protocol=HIGHEST_PROTOCOL)

    @staticmethod
    def unpickle(file: str) -> Any:
        with open(file, 'rb') as handle:
            return load(handle)

# 2. Константы

In [7]:
# MESSAGES_TO_MODELING_PARQUET_FILE = "../data/processed/messages_to_modeling.parquet"
# COMMENTS_TO_MODELING_PARQUET_FILE = "../data/processed/comments_to_modeling.parquet"

MESSAGES_TO_MODELING_PARQUET_FILE = "./drive/MyDrive/messages_to_modeling.parquet"
COMMENTS_TO_MODELING_PARQUET_FILE = "./drive/MyDrive/comments_to_modeling.parquet"

MODEL_NAME = 'DeepPavlov/rubert-base-cased'

# 3. Выгружаем данные

In [10]:
messages_to_modeling = pd.read_parquet(
    MESSAGES_TO_MODELING_PARQUET_FILE
)

print(messages_to_modeling.shape)
messages_to_modeling.sample(5)

(3473, 4)


,channel_name,channel_orientation,date,message
3726,bbbreaking,rus,2022-02-24 13:48:55+00:00,"️В Киеве объявлена воздушная тревога, власти п..."
14186,voynareal,ukr,2022-02-24 22:48:19+00:00,"️""Я спросил сегодня у 27 лидеров Европы будет ..."
1225,readovkanews,rus,2022-02-24 07:58:46+00:00,В сети появились видео с российскими ТОР в Хер...
22816,u_now,ukr,2022-02-24 05:46:54+00:00,Карта российских ударов по Украине
4613,breakingmash,rus,2022-03-07 10:43:49+00:00,Белгород. Колонна автобусов готова встречать б...


In [26]:
comments_to_modeling = pd.read_parquet(
    COMMENTS_TO_MODELING_PARQUET_FILE
)

print(comments_to_modeling.shape)
comments_to_modeling.sample(5)

(87947, 5)


,channel_name_x,channel_orientation_x,comment_date,comment_message,id
578,readovkanews,rus,2022-03-07 21:41:52+00:00,Господи 😂 Какой нахуй призрак Киева еблан хохл...,27644
62683,nexta_live,ukr,2022-03-07 15:40:04+00:00,Тоже самое было 1917,20347
59918,nexta_live,ukr,2022-03-07 19:12:56+00:00,"ооо, класический пример - существа с писюном п...",20375
58451,nexta_live,ukr,2022-03-07 19:00:19+00:00,Без этой машиной крови не прольётся кровь своб...,20353
45819,nexta_live,ukr,2022-03-07 20:53:52+00:00,Да и хрен с ними-мы переживем. С этими санкция...,20405


# 4. Получаем эмбеддинги

In [12]:
model = SentenceTransformer(MODEL_NAME)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/584 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 4.1. Посты

In [19]:
messages_to_encode = messages_to_modeling["message"].str.lower().str.replace(
    "\u200b", " ", regex=False
).str.strip().values

messages_embeddings = model.encode(
    messages_to_encode, 
    show_progress_bar=True
)

Batches:   0%|          | 0/109 [00:00<?, ?it/s]

In [22]:
messages_embeddings.shape

(3473, 768)

## 4.2. Комментарии

In [27]:
comments_to_encode = comments_to_modeling["comment_message"].str.lower().str.replace(
    "\u200b", " ", regex=False
).str.strip().values

comment_embeddings = model.encode(
    comments_to_encode, 
    show_progress_bar=True
)

Batches:   0%|          | 0/2749 [00:00<?, ?it/s]

In [28]:
comment_embeddings.shape

(87947, 768)

## 4.3. Выгружаем полноразмерные эмбеддинги
Будут лежать в `/data/processed`

In [30]:
Pickler.pickle(
    messages_embeddings, "./drive/MyDrive/first_stage_final_retrain/messages_to_modeling_embeddings.pkl"
)

Pickler.pickle(
    comment_embeddings, "./drive/MyDrive/first_stage_final_retrain/comments_to_modeling_embeddings.pkl"
)